In [ ]:
import pandas as pd
import openai
import time
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import json
import os

# OpenAI API key
openai.api_key = ''

Utils

In [ ]:
# Function to get embeddings
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-3-large",
        input=text
    )
    return response['data'][0]['embedding']

# convert row to JSON
def row_to_json(row):
    row_dict = row.to_dict()
    # Ensure all values are strings
    row_dict = {k: str(v) for k, v in row_dict.items()}
    
    return json.dumps(row_dict)

def find_most_similar_rows(row, df2, top_n=3):
    # Get the embedding of the specified row in df1
    row_embedding = get_embedding(row)
    df2_embeddings = np.array(df2['embeddings'].tolist())
    
    similarities = cosine_similarity([row_embedding], df2_embeddings)[0]
    df2['similarity'] = similarities
    
    # Sort by similarity
    most_similar = df2.sort_values(by=['similarity'], ascending=False).head(top_n)
    
    return most_similar

def print_full_json(series):
    for item in series:
        print(item)

Data Preparation

In [ ]:
df_plmdb = pd.read_excel('PLM-DB.xlsx', na_filter=False, dtype={'PartNumber': str})
df_qc = pd.read_excel('QC.xlsx', na_filter=False)

In [ ]:
# Pre-filters for QC
#...
# Pre-filters for PLM-DB

# Filter and rename columns
#...

# Define exclusion lists
#...

# Filter entries
#...

Generate a VectorDB from QC

In [ ]:
file_path = 'QC_with_embeddings.h5'

if os.path.exists(file_path):
    df_qc_filtered = pd.read_hdf(file_path, key='qc')
else:
    df_qc_filtered['json'] = df_qc_filtered.apply(row_to_json, axis=1)
    df_qc_filtered['embeddings'] = df_qc_filtered['json'].apply(get_embedding)

    # Save embeddings to JSON
    df_qc_filtered.to_hdf(file_path, key='qc', mode='w')

Search a Component in PLM-DB by Part Number (PN)

In [ ]:
plmdb_by_selected_pn = df_plmdb_filtered[df_plmdb_filtered['Part Number (PN)'] == '...']
print(plmdb_by_selected_pn)

Choose one of the component by its Index

In [ ]:
chosen_index = 32
selected_row_json = row_to_json(plmdb_by_selected_pn.loc[chosen_index])
print('Selected Component:')
print(selected_row_json)

Retrieve a list of n similar components from QC

In [ ]:
similar_components = find_most_similar_rows(selected_row_json, df_qc_filtered, 200)
print_full_json(similar_components['json'])

In [ ]:
prompt = f"""Given a Selected Component, identify the matching Similar Components based on the following rules:
- Direct Match: Same Part Number (PN), Package Code, SubPackage Code, and Manufacturer.
- Similarity Match: Same Package Code, SubPackage Code, and Manufacturer, but PN is different or not mentioned.
- Alternative Match: Same Package Code and Manufacturer, but different SubPackage Code.

Additional rules:
- In the Selected Component, the PN is found in the field 'Part Number (PN)', while in Similar Components, it is either found in the field 'Description' or 'Notes', usually within parenthesis, or not mentioned.
- For a Direct Match, the Part Number (PN) of the Selected Component must be exactly equal to the number found in the Similar Components.
- Package Codes written like 'AB1234-2', 'A1234', '1234', and '1234-I' are equivalent.
- Treat Manufacturers as equivalent if they share the same main brand name, regardless of additional descriptors or variations in wording. 

Output only a JSON with three keys: 'Direct', 'Similarity', and 'Alternative'. Each maps to a list of indexes of the matching Similar Components.

Selected Component: \"""
{row_to_json(plmdb_by_selected_pn.loc[chosen_index])}
\"""

Similar Components: \"""
"""

for index, row in similar_components.iterrows():
    prompt += f"Index {index}:{row_to_json(row)}\n"

prompt += '"""'

response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": "You are a highly accurate and detail-oriented expert system specialized in cross-referencing electronic components across heterogeneous databases. Always prioritize precision, consistency, adherence to the rules provided, and output results in JSON."},
            {"role": "user", "content": prompt}],
    max_tokens=2000,
    temperature=0,
    top_p=0
)

response = response.choices[0].message.content

# Parse JSON response
json_pattern = r'\{.*?\}'
match = re.search(json_pattern, response, re.DOTALL)
if match:
    parsed_response = json.loads(match.group(0))


In [ ]:
print('SELECTED COMPONENT:')
print(row_to_json(df_plmdb.loc[chosen_index]))
print('DIRECT QUALIFICATIONS (Same PN, Package, SubPackage, Manufacturer) FOR THE SELECTED COMPONENT:')
for i in parsed_response['Direct']:
        print(row_to_json(df_qc.loc[i]))
print('QUALIFICATIONS BY SIMILARITY (Same Package, Subpackage, and Manufacturer) FOR THE SELECTED COMPONENT:')
for i in parsed_response['Similarity']:
        print(row_to_json(df_qc.loc[i]))
print('POSSIBLE ALTERNATIVE QUALIFICATIONS (Same Package and Manufacturer) RELATIVE TO THE SELECTED COMPONENT:')
for i in parsed_response['Alternative']:
        print(row_to_json(df_qc.loc[i]))
print('OTHER RELEVANT QUALIFICATIONS:')
for component in similar_components.head(30).iterrows():
        if component[0] not in parsed_response['Direct'] and component[0] not in parsed_response['Similarity'] and component[0] not in parsed_response['Alternative']:
                print(row_to_json(df_qc.loc[component[0]]))

In [ ]:
# Get ground truth
gt_direct = pd.read_csv('Direct Qualifications_Ground Truth.csv')
gt_sim = pd.read_csv('Qualifications by Similarity_Ground Truth.csv')
gt_alt = pd.read_csv('Qualifications Alternative_Ground Truth.csv')

In [ ]:
product_numbers = pd.concat([gt_direct['PN'], gt_sim['PN'], gt_alt['PN']]).drop_duplicates().tolist()

In [ ]:
if os.path.exists('results.json'):
    # If the file exists, load the results from the file
    with open('results.json', 'r') as f:
        results = json.load(f)
else:
    # If the file does not exist, initialize an empty list
    results = []

In [ ]:
for product_number in product_numbers:

    if any(product_number in result for result in results):
        continue  # Skip this iteration if product_number is already in results

    plmdb_by_pns = df_plmdb_filtered[df_plmdb_filtered['Part Number (PN)'] == product_number]

    result_for_pn = {}
    for i, component in plmdb_by_pns.iterrows():
        similar_q = find_most_similar_rows(row_to_json(component), df_qc_filtered, 200)

        prompt = f"""Given a Selected Component, identify the matching Similar Components based on the following rules:
- Direct Match: Same Part Number (PN), Package Code, SubPackage Code, and Manufacturer.
- Similarity Match: Same Package Code, SubPackage Code, and Manufacturer, but PN is different or not mentioned.
- Alternative Match: Same Package Code and Manufacturer, but different SubPackage Code.

Additional rules:
- In the Selected Component, the PN is found in the field 'Part Number (PN)', while in Similar Components, it is either found in the field 'Description' or 'Notes', usually within parenthesis, or not mentioned.
- For a Direct Match, the Part Number (PN) of the Selected Component must be exactly equal to the number found in the Similar Components.
- For Package Codes, focus only on the main number. Package Codes written like 'AB1234-2', 'A1234', '1234', and '1234-I' are equivalent.
- Treat Manufacturers as equivalent if they share the same main brand name, regardless of additional descriptors or variations in wording. 

Output only a JSON with three keys: 'Direct', 'Similarity', and 'Alternative'. Each maps to a list of indexes of the matching Similar Components.

Selected Component: \"""
{row_to_json(component)}
\"""

Similar Components:
"""

        for index, row in similar_q.iterrows():
            prompt += f"Index {index}:{row_to_json(row)}\n"

        prompt += '"""'
        
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "You are a highly accurate and detail-oriented expert system specialized in cross-referencing electronic components across heterogeneous databases. Always prioritize precision, consistency, adherence to the rules provided, and output results in JSON."},
                    {"role": "user", "content": prompt}],
            max_tokens=2000,
            temperature=0,
            top_p=0
        )

        #time.sleep(31) # To avoid rate_limit errors

        response = response.choices[0].message.content

        # Parse JSON response
        json_pattern = r'\{.*?\}'
        match = re.search(json_pattern, response, re.DOTALL)
        if match:
            parsed_response = json.loads(match.group(0))
        else:
            print('Error Parsing JSON')

        # Substitute index with Q_Number
        for key in parsed_response:
            parsed_response[key] = [df_qc.loc[i]['Number'] for i in parsed_response[key]]

        # Merge with other components of the same PN
        for key in parsed_response:
            result_for_pn[key] = result_for_pn.get(key, []) + parsed_response.get(key, [])
    
    results.append({product_number: result_for_pn})

In [ ]:
# Save the 'results' list to a JSON file
with open('results.json', 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
true_positives_direct = 0
true_positives_sim = 0
true_positives_alt = 0

# Ground truth and predicted counts
direct_qualifications_gt_count = 0
sim_qualifications_gt_count = 0
alt_qualifications_gt_count = 0

direct_qualifications_pred_count = 0
sim_qualifications_pred_count = 0
alt_qualifications_pred_count = 0

all_qualifications_pred_count = 0
all_qualifications_gt_count = 0

# Initialize Jaccard Scores
jaccard_direct = 0
jaccard_sim = 0
jaccard_alt = 0
jaccard_all = 0

for component in results:
    # Direct qualification
    direct_qualifications_pred = list(component.values())[0]['Direct']
    direct_qualifications_gt = list(gt_direct[gt_direct['PN'] == list(component.keys())[0]]['QN'])
    direct_intersection = len(set(direct_qualifications_pred) & set(direct_qualifications_gt))
    direct_union = len(set(direct_qualifications_pred) | set(direct_qualifications_gt))

    true_positives_direct += direct_intersection
    direct_qualifications_gt_count += len(set(direct_qualifications_gt))
    direct_qualifications_pred_count += len(set(direct_qualifications_pred))

    # Update Jaccard for Direct
    if direct_union > 0:
        jaccard_direct += direct_intersection / direct_union
    else:
        if direct_intersection == 0:
            jaccard_direct += 1
        else:
            jaccard_direct += 0

    # Qualification by similarity
    sim_qualifications_pred = list(component.values())[0]['Similarity']
    sim_qualifications_gt = list(gt_sim[gt_sim['PN'] == list(component.keys())[0]]['QN'])
    sim_intersection = len(set(sim_qualifications_pred) & set(sim_qualifications_gt))
    sim_union = len(set(sim_qualifications_pred) | set(sim_qualifications_gt))

    true_positives_sim += sim_intersection
    sim_qualifications_gt_count += len(set(sim_qualifications_gt))
    sim_qualifications_pred_count += len(set(sim_qualifications_pred))

    # Update Jaccard for similarity
    if sim_union > 0:
        jaccard_sim += sim_intersection / sim_union
    else:
        if sim_intersection == 0:
            jaccard_sim += 1
        else:
            jaccard_sim += 0

    # Qualification (alternative)
    alt_qualifications_pred = list(component.values())[0]['Alternative']
    alt_qualifications_gt = list(gt_alt[gt_alt['PN'] == list(component.keys())[0]]['QN'])
    alt_intersection = len(set(alt_qualifications_pred) & set(alt_qualifications_gt))
    alt_union = len(set(alt_qualifications_pred) | set(alt_qualifications_gt))

    true_positives_alt += alt_intersection
    alt_qualifications_gt_count += len(set(alt_qualifications_gt))
    alt_qualifications_pred_count += len(set(alt_qualifications_pred))

    # Update Jaccard for Alternative
    if alt_union > 0:
        jaccard_alt += alt_intersection / alt_union
    else:
        if alt_intersection == 0:
            jaccard_alt += 1
        else:
            jaccard_alt += 0

    # Compute overall qualifications
    all_qualifications_pred = list(set(direct_qualifications_pred) | set(sim_qualifications_pred) | set(alt_qualifications_pred))
    all_qualifications_gt = list(set(direct_qualifications_gt) | set(sim_qualifications_gt) | set(alt_qualifications_gt))
    all_intersection = len(set(all_qualifications_pred) & set(all_qualifications_gt))
    all_union = len(set(all_qualifications_pred) | set(all_qualifications_gt))

    # Update overall counts
    all_qualifications_pred_count += len(all_qualifications_pred)
    all_qualifications_gt_count += len(all_qualifications_gt)

    # Update Jaccard for All
    if all_union > 0:
        jaccard_all += all_intersection / all_union
    else:
        if all_intersection == 0:
            jaccard_all += 1
        else:
            jaccard_all += 0

# Calculate overall precision, recall, and F1
precision = (true_positives_direct + true_positives_sim + true_positives_alt) / all_qualifications_pred_count
recall = (true_positives_direct + true_positives_sim + true_positives_alt) / all_qualifications_gt_count
f1_score = 2 * (precision * recall) / (precision + recall)

# Jaccard Scores
jaccard_direct /= len(results)
jaccard_sim /= len(results)
jaccard_alt /= len(results)
jaccard_all /= len(results)

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)
print('Jaccard All:', jaccard_all)
print("------------------------------------")

# Direct metrics
precision_direct = true_positives_direct / direct_qualifications_pred_count
recall_direct = true_positives_direct / direct_qualifications_gt_count
f1_direct = 2 * (precision_direct * recall_direct) / (precision_direct + recall_direct)

print('Precision_direct:', precision_direct)
print('Recall_direct:', recall_direct)
print('F1-score_direct:', f1_direct)
print('Jaccard_direct:', jaccard_direct)
print("------------------------------------")

# Similarity metrics
precision_sim = true_positives_sim / sim_qualifications_pred_count
recall_sim = true_positives_sim / sim_qualifications_gt_count
f1_sim = 2 * (precision_sim * recall_sim) / (precision_sim + recall_sim)

print('Precision_sim:', precision_sim)
print('Recall_sim:', recall_sim)
print('F1-score_sim:', f1_sim)
print('Jaccard_sim:', jaccard_sim)
print("------------------------------------")

# Alternative metrics
precision_alt = true_positives_alt / alt_qualifications_pred_count
recall_alt = true_positives_alt / alt_qualifications_gt_count
f1_alt = 2 * (precision_alt * recall_alt) / (precision_alt + recall_alt)

print('Precision_alt:', precision_alt)
print('Recall_salt:', recall_alt)
print('F1-score_alt:', f1_alt)
print('Jaccard_alt:', jaccard_alt)


Recall within top 200

In [ ]:
all_qualifications_gt_count = 0
all_qualifications_pred_count = 0
for product_number in product_numbers:
    plmdb_by_pns = df_plmdb_filtered[df_plmdb_filtered['Part Number (PN)'] == product_number]

    similar_q_pn = set()
    for i, component in plmdb_by_pns.iterrows():
        similar_q = find_most_similar_rows(row_to_json(component), df_qc_filtered, 200).index
        similar_q = [df_qc.loc[index]['Number'] for index in similar_q]
        similar_q_pn = set(similar_q) | similar_q_pn
    
    direct_qualifications_gt = list(gt_direct[gt_direct['PN'] == product_number]['QN'])
    sim_qualifications_gt = list(gt_sim[gt_sim['PN'] == product_number]['QN'])
    alt_qualifications_gt = list(gt_alt[gt_alt['PN'] == product_number]['QN'])
    all_qualifications_gt = set(direct_qualifications_gt) | set(sim_qualifications_gt) | set(alt_qualifications_gt)
    
    all_qualifications_pred = similar_q_pn & all_qualifications_gt

    all_qualifications_gt_count += len(all_qualifications_gt)
    all_qualifications_pred_count += len(all_qualifications_pred)
    
print("Recall from Top-n Similar:")  # n = 200
print(all_qualifications_pred_count/all_qualifications_gt_count)